In [16]:
import scipy.io
import numpy as np
np.set_printoptions(linewidth=200)
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [17]:
from Encoding import load
dataset_name = "N050_Mono"
par_data, bc = load(dataset_name,data_type="par_data"), load(dataset_name,data_type="bc")

In [18]:
par = par_data[0][0]
bc_t = bc[0]

In [19]:
from Encoding import ProjectPointsToCylinder, ProjectPointsToHyperplane, ConstructTopology
top = ConstructTopology(par,bc_t,0.1)-1

In [20]:
from Encoding import TopologySlice
bc[0]

array([[[0.74604114, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.79485544, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 1.12686156, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0],
        [-1.16630084, -1.19855435, -0.40231877, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [-1.16630084, -1.19855435, -0.40231877, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
        [-1.16630084, -1.19855435, -0.40231877, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]],

       [[-0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, -0.002293770495, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.002293770495, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]], dtype=object)

In [21]:
def GetVirtualParticlesCoords(par_step,top_step,bc_step):
    P_wall,P_W_particles = [],[]
    for wall_id,wall in enumerate(bc_step[0]):
        if wall[-1] == 1:
            ProjectionFunction = ProjectPointsToHyperplane
        if wall[-1] == 0:
            ProjectionFunction = ProjectPointsToHyperplane

        P_W_mask = -top_step[:,1]-2==wall_id
        P_W_idx = top_step[P_W_mask]
        P_W_particles_temp = par_step[P_W_idx[:,0],:][:,:3]
        P_wall.append(ProjectionFunction(P_W_particles_temp,wall))
        P_W_particles.append(P_W_particles_temp)

    P_wall = np.astype(np.concatenate(P_wall),float)
    P_W_particles = np.astype(np.concatenate(P_W_particles),float)

    rel_coord = P_W_particles-P_wall
    normal_vectors = rel_coord/np.linalg.norm(rel_coord,axis=1,keepdims=True)
    return P_wall, normal_vectors

def BCEncoding2(par_step,top_step,bc_step):
    
    P_wall, normal_vectors = GetVirtualParticlesCoords(par_step,top_step,bc_step)

    P_virtual = np.concatenate((P_wall,
                                np.zeros((P_wall.shape[0],3)),                      
                                np.astype(normal_vectors,int),
                                np.zeros((P_wall.shape[0],1))),                      
                                axis=1)

    top_new = top_step.copy()
    n_new = P_virtual.shape[0]
    for i in range(n_new):
        top_new[-n_new+i,1] = par_step.shape[0]+i
    
    return P_virtual, top_new

P_virtual, top_new = BCEncoding2(par_data[0][0],top,bc_t)



In [24]:
P_virtual, top_new

(array([[ 0.74604114, -0.31352618,  0.83830016,  0.        ,  0.        ,  0.        , -1.        ,  0.        ,  0.        ,  0.        ],
        [-0.26917921,  0.79485544,  0.40080929,  0.        ,  0.        ,  0.        ,  0.        , -1.        ,  0.        ,  0.        ],
        [-0.32395348,  0.51613544,  1.12686156,  0.        ,  0.        ,  0.        ,  0.        ,  0.        , -1.        ,  0.        ],
        [-0.50620103,  0.22057354,  1.12686156,  0.        ,  0.        ,  0.        ,  0.        ,  0.        , -1.        ,  0.        ],
        [-0.3079721 , -0.12162749,  1.12686156,  0.        ,  0.        ,  0.        ,  0.        ,  0.        , -1.        ,  0.        ],
        [-0.86901912, -0.39969723,  1.12686156,  0.        ,  0.        ,  0.        ,  0.        ,  0.        , -1.        ,  0.        ],
        [-0.08986017, -0.92775361,  1.12686156,  0.        ,  0.        ,  0.        ,  0.        ,  0.        , -1.        ,  0.        ],
        [-0.39151132

In [23]:
point_on_wall = ProjectPointsToCylinder(particles,cyl)
#point_on_wall = ProjectPointsToHyperplane(particles,wall)


NameError: name 'cyl' is not defined

In [ ]:
from Plotting import Plot3DVectors
#fig = plt.figure()
#ax = fig.add_subplot(projection='3d')
#Plot3DVectors(ax,axis_projection,s)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
vector = P_virtual[:,:3]
x = vector[:,0]
y = vector[:,1]
z = vector[:,2]
ax.scatter(x,y,z)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
vector = point_on_wall
x = vector[:,0]
y = vector[:,1]
z = vector[:,2]
ax.scatter(x,y,z)
vector = particles
x = vector[:,0]
y = vector[:,1]
z = vector[:,2]
ax.scatter(x,y,z)

In [ ]:
vector = point_on_wall
x = vector[:,0]
y = vector[:,1]
z = vector[:,2]
plt.plot(y,z,'o',color='b')
vector = particles
x = vector[:,0]
y = vector[:,1]
z = vector[:,2]
plt.plot(y,z,'o',color='r')